In [1]:
#V14 changes:
    # organized section #0
    # section #1 -> updated LABEL_NAMES to include contempt and unknown
    # section #2 -> added save_model_and_processor function
    # section #6 -> reworked perceptual clustering to include these labels:
        # disgust, sadness, fear, questioning, contempt
    # section #7 -> updated targeted augmentation for new labels except unknown
    # section #8 -> added contempt to minority_classes_names
        #updated minority_classes_names and minority_classes
    # sction #11 -> updated to exclude unknown
    # section #15 deleted -> renumbered rescue section to #15

In [2]:
# --------------------------
# 0. Imports
# --------------------------
# Standard Library Imports
import gc
import glob
import multiprocessing as mp
import os
import random
import shutil
import subprocess
import sys
import time

# Third-Party Imports
import accelerate
import dill
import face_recognition
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
#import tensorflow as tf
import torch
import torch.nn.functional as F
import torchvision.transforms as T
import transformers

# From Imports
from collections import Counter
from datasets import ClassLabel, Dataset, Features, Image as DatasetsImage, concatenate_datasets, load_dataset
from datetime import datetime
from functools import partial
from imagehash import phash
from io import BytesIO
from pathlib import Path
from PIL import Image, ImageOps, ExifTags, UnidentifiedImageError
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, log_loss
from torch import nn
from torch.nn import functional as F
from torch.optim import AdamW, LBFGS
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torchvision import transforms
from torchvision.transforms import (
    GaussianBlur,
    RandAugment,
    RandomAffine,
    RandomApply,
    ToPILImage,
    ToTensor
)
from tqdm import tqdm
from transformers import (
    AutoImageProcessor,
    AutoModelForImageClassification,
    EarlyStoppingCallback,
    TrainingArguments,
    Trainer,
)

In [3]:
# --------------------------
# 1. Global Configurations
# --------------------------
RUN_INFERENCE = True  # Toggle this off to disable running inference
IMAGE_DIR = "/Users/natalyagrokh/AI/ml_expressions/img_datasets/ferckjalfag_dataset"
BASE_PATH = IMAGE_DIR

LABEL_NAMES = [
    'anger', 'disgust', 'fear', 'happiness', 'neutral',
    'questioning', 'sadness', 'surprise', 'contempt', 'unknown'
]
id2label = dict(enumerate(LABEL_NAMES))
label2id = {v: k for k, v in id2label.items()}

VALID_EXTENSIONS = (".jpg", ".jpeg", ".png", ".tif", ".tiff")

def is_valid_image(filename):
    return filename.lower().endswith(VALID_EXTENSIONS) and not filename.startswith("._")

label_mapping = {name.lower(): name for name in LABEL_NAMES}

# 🔢 Dynamically determine the next version
def get_next_version(base_dir):

    # Use glob to find all entries matching the pattern
    all_entries = glob.glob(os.path.join(base_dir, "V*_*"))
    
    # Filter to include only directories
    existing = [
        os.path.basename(d) for d in all_entries if os.path.isdir(d)
    ]

    # Extract version numbers from the directory names
    versions = [
        int(d[1:].split("_")[0]) for d in existing
        if d.startswith("V") and "_" in d and d[1:].split("_")[0].isdigit()
    ]
    
    # Determine the next version number
    next_version = max(versions, default=0) + 1
    return f"V{next_version}"

# Automatically create a versioned output folder
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
VERSION = get_next_version("/Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training")
VERSION_TAG = VERSION + "_" + timestamp
SAVE_DIR = os.path.join("/Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training", VERSION_TAG)
LOGITS_PATH = os.path.join(SAVE_DIR, f"logits_eval_{VERSION}.npy")
LABELS_PATH = os.path.join(SAVE_DIR, f"labels_eval_{VERSION}.npy")
os.makedirs(SAVE_DIR, exist_ok=True)
print(f"📁 Output directory created: {SAVE_DIR}")

📁 Output directory created: /Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V14_20250614_190959


In [4]:
# --------------------------
# 2. Utility Functions (Metrics & Calibration)
# --------------------------

# 🔍 Compute perceptual hash for image similarity clustering (used in REVIEW and Disgust curation)
def compute_hash(image_path):
    try:
        img = Image.open(image_path).convert("L").resize((64, 64))
        return str(phash(img))
    except Exception:
        return None

# 🔄 Smoothed Cross Entropy Loss (ε = 0.05)
class SmoothedCrossEntropyLoss(nn.Module):
    def __init__(self, smoothing=0.05):
        super().__init__()
        self.smoothing = smoothing

    def forward(self, logits, target):
        num_classes = logits.size(1)
        with torch.no_grad():
            smooth_labels = torch.full_like(logits, self.smoothing / (num_classes - 1))
            smooth_labels.scatter_(1, target.unsqueeze(1), 1.0 - self.smoothing)
        log_probs = F.log_softmax(logits, dim=1)
        loss = -(smooth_labels * log_probs).sum(dim=1).mean()
        return loss

# ⚠️ Confidence Penalty to Reduce Overconfidence
def confidence_penalty(logits, beta=0.05):
    probs = F.softmax(logits, dim=1)
    log_probs = F.log_softmax(logits, dim=1)
    entropy = -torch.sum(probs * log_probs, dim=1)
    return beta * entropy.mean()

# 📊 Compute Metrics with Confusion Matrix Logging
def compute_metrics_with_confusion(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    print("\nClassification Report:")
    print(classification_report(labels, preds, target_names=LABEL_NAMES))

    # Save raw values for calibration or further use
    np.save(os.path.join(SAVE_DIR, f"logits_eval_{VERSION}.npy"), logits)
    np.save(os.path.join(SAVE_DIR, f"labels_eval_{VERSION}.npy"), labels)

    # Generate confusion matrix heatmap
    cm = confusion_matrix(labels, preds)
    plt.figure(figsize=(8, 6))
    sns.heatmap(
        cm, annot=True, fmt="d", cmap="Blues",
        xticklabels=LABEL_NAMES,
        yticklabels=LABEL_NAMES
    )
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title("Confusion Matrix")
    plt.tight_layout()
    plt.savefig(os.path.join(SAVE_DIR, f"confusion_matrix_epoch_{VERSION}.png"))
    plt.close()

    # Identify top confused class pairs (excluding diagonal)
    confusion_pairs = [
        ((LABEL_NAMES[i], LABEL_NAMES[j]), cm[i][j])
        for i in range(len(LABEL_NAMES))
        for j in range(len(LABEL_NAMES)) if i != j
    ]
    top_confusions = sorted(confusion_pairs, key=lambda x: x[1], reverse=True)[:3]
    print("\nTop 3 confused class pairs:")
    for (true_label, pred_label), count in top_confusions:
        print(f"  - {true_label} → {pred_label}: {count} instances")

    # Compute average prediction entropy
    softmax_probs = F.softmax(torch.tensor(logits), dim=-1)
    entropies = -torch.sum(softmax_probs * torch.log(softmax_probs + 1e-12), dim=-1)
    avg_entropy = entropies.mean().item()
    print(f"\n🧠 Avg prediction entropy: {avg_entropy:.4f}")

    # Entropy per class (sorted by entropy)
    entropy_per_class = []
    for idx, class_name in enumerate(LABEL_NAMES):
        mask = (np.array(labels) == idx)
        if mask.any():
            class_entropy = entropies[mask].mean().item()
            entropy_per_class.append((class_name, class_entropy))
    entropy_per_class.sort(key=lambda x: x[1], reverse=True)

    print("\n🔍 Class entropies (sorted):")
    for class_name, entropy in entropy_per_class:
        print(f"  - {class_name}: entropy = {entropy:.4f}")

    accuracy = (preds == labels).mean()
    return {"accuracy": accuracy}


# 🌡️ Apply Temperature Scaling for Calibration
def apply_temperature_scaling(logits_path, labels_path):
    if not (os.path.exists(logits_path) and os.path.exists(labels_path)):
        print(f"❌ Missing files:\n  - {logits_path if not os.path.exists(logits_path) else ''}\n - {labels_path if not os.path.exists(labels_path) else ''}")
        return None

    print(f"📂 Loading logits from: {logits_path}")
    print(f"📂 Loading labels from: {labels_path}")

    logits = torch.tensor(np.load(logits_path), dtype=torch.float32).to(device)
    labels = torch.tensor(np.load(labels_path), dtype=torch.long).to(device)

    class TemperatureScaler(nn.Module):
        def __init__(self):
            super().__init__()
            self.temperature = nn.Parameter(torch.ones(1) * 1.5)

        def forward(self, logits):
            return logits / self.temperature

    model = TemperatureScaler().to(device)
    optimizer = LBFGS([model.temperature], lr=0.01, max_iter=50)

    def eval_fn():
        optimizer.zero_grad()
        loss = F.cross_entropy(model(logits), labels)
        loss.backward()
        return loss

    optimizer.step(eval_fn)
    calibrated_logits = model(logits)
    probs = F.softmax(calibrated_logits, dim=1).detach().cpu().numpy()
    logloss = log_loss(labels.cpu().numpy(), probs)

    # Save optimal temperature
    temperature_value = model.temperature.item()
    torch.save(
        torch.tensor([temperature_value]),
        os.path.join(SAVE_DIR, f"{VERSION}_calibrated_temperature.pt")
    )
    print(f"✅ Optimal temperature: {temperature_value:.4f}")
    print(f"✅ Calibrated Log Loss: {logloss:.4f}")
    return temperature_value, logits.cpu(), labels.cpu()


# 📈 Plot Reliability Diagram (Calibration Curve)
def plot_reliability_diagram(logits, labels, temperature, n_bins=15):
    probs = F.softmax(logits / temperature, dim=1)
    confidences, predictions = torch.max(probs, 1)
    accuracies = predictions.eq(labels)

    bins = torch.linspace(0, 1, n_bins + 1)
    bin_lowers, bin_uppers = bins[:-1], bins[1:]

    bin_accuracies, bin_confidences = [], []
    for lower, upper in zip(bin_lowers, bin_uppers):
        mask = (confidences > lower) & (confidences <= upper)
        if mask.any():
            bin_accuracies.append(accuracies[mask].float().mean())
            bin_confidences.append(confidences[mask].mean())

    plt.figure(figsize=(6, 6))
    plt.plot(bin_confidences, bin_accuracies, marker='o', label='Model')
    plt.plot([0, 1], [0, 1], linestyle='--', label='Perfect Calibration')
    plt.title("Reliability Diagram (After Temperature Scaling)")
    plt.xlabel("Confidence")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    output_path = os.path.join(SAVE_DIR, f"{VERSION}_reliability_diagram_calibrated.png")
    plt.savefig(output_path)
    plt.close()
    print(f"📊 Saved reliability diagram to {output_path}")

# saving model and processor
def save_model_and_processor(model, processor, save_dir, trainer=None):
    print(f"Saving model and processor to: {save_dir}")
    
    model = model.to("cpu")

    # Save processor
    processor.save_pretrained(save_dir)
    print(f"✅ Processor saved to: {SAVE_DIR}")
    
    # Save full model
    model.save_pretrained(SAVE_DIR, safe_serialization=True)
    print(f"✅ Full model saved to: {SAVE_DIR}")

    # Save state dict
    final_model_path = os.path.join(SAVE_DIR, 'final_model.pth')
    torch.save(model.state_dict(), final_model_path)
    print(f"✅ State dict saved to: {final_model_path}")

    # Save trainer state
    if trainer is not None:
        try:
            trainer.save_model(os.path.join(save_dir, "backup_trainer_model"))
            print("✅ Trainer backup saved.")
        except Exception as e:
            print(f"⚠️ Failed to save trainer backup: {e}")

    # Memory cleanup
    del model
    gc.collect()
    try:
        torch.cuda.empty_cache()
    except Exception:
        pass  # Not all systems have CUDA
    print("✅ Memory cleanup complete after save.")

In [5]:
# --------------------------
# 3. Auto-Load Latest Pretrained Model and Processor
# --------------------------

# Automatically load latest model path
MODEL_ROOT = "/Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training"
# List all version folders in descending order
model_dirs = sorted(
    [os.path.join(MODEL_ROOT, d) for d in os.listdir(MODEL_ROOT)
     if d.startswith("V") and os.path.isdir(os.path.join(MODEL_ROOT, d))],
    key=lambda x: os.path.getmtime(x),
    reverse=True
)

# Remove the current output version (to avoid loading from empty target)
model_dirs = [d for d in model_dirs if VERSION in d or not d.startswith(VERSION)]
model_dirs = [d for d in model_dirs if os.path.basename(d).startswith("V") and d != SAVE_DIR]

# Pick the most recent complete model (not current output)
if len(model_dirs) < 1:
    raise FileNotFoundError("❌ No earlier model folders found.")
model_path = model_dirs[0]
print(f"✅ Auto-loaded model from: {model_path}")

# Load base model and processor
model = AutoModelForImageClassification.from_pretrained(model_path)
processor = AutoImageProcessor.from_pretrained(model_path)

# Modify classification head with Dropout for regularization
model.classifier = nn.Sequential(
    nn.Dropout(p=0.1),
    nn.Linear(model.classifier.in_features, len(id2label))
)

# Replace classification head to match current label schema
model.config.id2label = id2label
model.config.label2id = label2id
model.config.num_labels = len(LABEL_NAMES)

# Define device and push model to device
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print("🖥️ Using device:", device)
model.to(device).eval()

Some weights of the model checkpoint at /Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V13_20250527_161430 were not used when initializing ViTForImageClassification: ['classifier.1.bias', 'classifier.1.weight']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at /Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V13_20250527_161430 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model 

✅ Auto-loaded model from: /Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V13_20250527_161430
🖥️ Using device: mps


ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)
            (intermed

In [6]:
# --------------------------
# 4. Load and Prepare Dataset
# --------------------------
dataset = load_dataset(
    "imagefolder",
    data_dir="/Users/natalyagrokh/AI/ml_expressions/img_datasets/ferckjalfag_dataset",
    split="train",
    cache_dir="/tmp/hf_cache"
)

counter = {"n": 0}

def reconcile_labels(example):
    counter["n"] += 1
    if counter["n"] % 1000 == 0:
        print(f"Processed {counter['n']} images...")

    label = example.get("label", None)

    if isinstance(label, int):
        original_label = dataset.features["label"].int2str(label).strip().lower()
    elif isinstance(label, str):
        original_label = label.strip().lower()
    else:
        file_path = getattr(example["image"], "filename", None)
        original_label = os.path.basename(os.path.dirname(file_path)).lower() if file_path else None

    pretrain_label = label_mapping.get(original_label)
    example["label"] = label2id[pretrain_label] if pretrain_label is not None else -1
    return example

# Single-threaded labeling to preserve .filename
dataset = dataset.map(reconcile_labels, desc="Re-labeling dataset")
dataset = dataset.filter(lambda x: x["label"] != -1)

print(f"✅ Total examples after filtering: {len(dataset)}")

Resolving data files:   0%|          | 0/17630 [00:00<?, ?it/s]

✅ Total examples after filtering: 17619


In [7]:
# --------------------------
# 5. Dataset Label Overview and Folder Stats
# --------------------------
def analyze_dataset_structure(dataset, id2label, base_path):
    # Print label schema from the dataset
    print("Label schema (from dataset):", dataset.features["label"])

    # Label distribution from the dataset object
    label_counts = Counter(dataset["label"])
    print("\n📊 Full dataset label distribution (from Dataset object):")
    for label_id, count in sorted(label_counts.items()):
        print(f"  {id2label[label_id]}: {count} examples")

    # Dynamically detect minority classes (lowest 3 frequencies)
    N = 3
    minority_classes = set(
        label for label, _ in sorted(label_counts.items(), key=lambda x: x[1])[:N]
    )
    print(f"\n⚠️  Dynamically identified minority classes: {[id2label[i] for i in minority_classes]}")

    # Count images per directory, and store for later validation
    folder_image_counts = {}
    print("\n📂 Image count per label folder:")
    for label in sorted(os.listdir(base_path)):
        label_path = os.path.join(base_path, label)
        if os.path.isdir(label_path):
            valid_images = [img for img in os.listdir(label_path) if is_valid_image(img)]
            folder_image_counts[label] = len(valid_images)
            print(f"  {label}: {len(valid_images)} images")

    return minority_classes, folder_image_counts

# Example usage right after dataset loading
minority_classes, folder_image_counts = analyze_dataset_structure(dataset, id2label, BASE_PATH)

Label schema (from dataset): ClassLabel(names=['anger', 'contempt', 'disgust', 'fear', 'happiness', 'neutral', 'questioning', 'sadness', 'surprise', 'unknown'], id=None)

📊 Full dataset label distribution (from Dataset object):
  anger: 2302 examples
  disgust: 309 examples
  fear: 1432 examples
  happiness: 2892 examples
  neutral: 3334 examples
  questioning: 1943 examples
  sadness: 1706 examples
  surprise: 2779 examples
  contempt: 536 examples
  unknown: 386 examples

⚠️  Dynamically identified minority classes: ['contempt', 'disgust', 'unknown']

📂 Image count per label folder:
  anger: 2302 images
  contempt: 536 images
  disgust: 309 images
  fear: 1432 images
  happiness: 2892 images
  neutral: 3334 images
  questioning: 1943 images
  sadness: 1706 images
  surprise: 2779 images
  unknown: 386 images


In [8]:
# --------------------------
# 6. Perceptual Clustering for Ambiguous/Confused Classes
# --------------------------

CLUSTER_TARGETS = ["disgust", "sadness", "fear", "questioning", "contempt"]

for class_name in CLUSTER_TARGETS:
    class_dir = os.path.join(BASE_PATH, class_name)
    if not os.path.isdir(class_dir):
        print(f"⚠️ Class dir not found: {class_dir} (skipping)")
        continue

    class_images = [
        os.path.join(class_dir, f) for f in os.listdir(class_dir)
        if is_valid_image(f)
    ]
    hash_map = {}
    for path in class_images:
        h = compute_hash(path)
        if h:
            hash_map.setdefault(h, []).append(path)

    cluster_dir = os.path.join(SAVE_DIR, f"{class_name}_clusters")
    os.makedirs(cluster_dir, exist_ok=True)

    print(f"🔍 {class_name.capitalize()} hash clusters with more than 1 image:")
    for h, paths in hash_map.items():
        if len(paths) > 1:
            cluster_path = os.path.join(cluster_dir, h)
            os.makedirs(cluster_path, exist_ok=True)
            for p in paths:
                shutil.copy(p, cluster_path)
            print(f"  - Cluster {h[:8]}: {len(paths)} images copied for review")

🔍 Disgust hash clusters with more than 1 image:
🔍 Sadness hash clusters with more than 1 image:
  - Cluster 958c52e1: 2 images copied for review
  - Cluster ee9a8d33: 2 images copied for review
  - Cluster d0890396: 2 images copied for review
  - Cluster bb0d06f2: 2 images copied for review
  - Cluster d7f00fa2: 2 images copied for review
🔍 Fear hash clusters with more than 1 image:
  - Cluster 9ae56592: 2 images copied for review
  - Cluster 91c8ee81: 2 images copied for review
  - Cluster dae5a596: 2 images copied for review
🔍 Questioning hash clusters with more than 1 image:
  - Cluster da014886: 2 images copied for review
  - Cluster 9db42783: 2 images copied for review
🔍 Contempt hash clusters with more than 1 image:


In [9]:
# --------------------------
# 7. Class Frequency-Aware Augmentation Targeting
# --------------------------

# Compute label frequencies from train split (post filtering)
label_freqs = Counter(dataset["label"])
label_id2name = {v: k for k, v in label2id.items()}
label_name2id = {v: k for k, v in label_id2name.items()}

# Get lowest-count classes dynamically
minority_by_count = sorted(label_freqs, key=label_freqs.get)[:3]
minority_by_name = [label_id2name[i] for i in minority_by_count]
minority_by_name = [n for n in minority_by_name if n != "unknown"]

# Manually include known confused or underperforming classes
manual_focus_classes = ['disgust', 'questioning', 'contempt']

# Merge and deduplicate
minority_class_names = list(set(minority_by_name + manual_focus_classes))

# Final list as label indices
minority_classes = [label_name2id[name] for name in minority_class_names]

print(f"🎯 Targeted minority augmentation will apply to: {minority_class_names}")

🎯 Targeted minority augmentation will apply to: ['contempt', 'disgust', 'questioning']


In [12]:
# --------------------------
# 8. Define Data Augmentation and Preprocessing Transformation
# --------------------------

# Baseline augmentation
data_augment = T.Compose([
    T.RandomHorizontalFlip(),
    T.RandomRotation(10),
    T.ColorJitter(brightness=0.1, contrast=0.1)
])

# RandAugment for specific minority classes only
minority_classes_names = minority_class_names
minority_classes = [label2id[label] for label in minority_classes_names]

minority_aug = T.Compose([
    RandAugment(num_ops=2, magnitude=9),
    T.RandomResizedCrop(224, scale=(0.7, 1.0)),
    T.ColorJitter(0.3, 0.3, 0.3, 0.1),
])

# Augmentation counter tracking
aug_count = Counter()

def make_transform_function(processor, minority_classes):
    def transform_function(example):
        label = example["label"]
        aug_pipeline = minority_aug if label in minority_classes else data_augment
        aug_count[label] += 1

        if example["image"].mode != "RGB":
            example["image"] = example["image"].convert("RGB")

        augmented_image = aug_pipeline(example["image"])
        inputs = processor(augmented_image, return_tensors="pt")
        inputs = {k: v.squeeze(0) for k, v in inputs.items()}
        inputs["labels"] = example["label"]
        return inputs
    return transform_function

# After mapping finishes:
dataset = dataset.map(make_transform_function(processor, minority_classes))
formatted_counts = {LABEL_NAMES[k]: v for k, v in aug_count.items()}
print(f"✅ Augmentation counts: {formatted_counts}")

# Explicitly log dataset snapshots (class distribution) to a 
# CSV or JSON after each run for easy future diffing and tracking
snapshot_path = os.path.join(SAVE_DIR, f"{VERSION}_augmentation_snapshot.csv")
aug_snapshot = pd.DataFrame.from_dict(dict(aug_count), orient='index', columns=['count'])
aug_snapshot.to_csv(snapshot_path)

print(f"✅ Saved augmentation snapshot to {snapshot_path}")

Map:   0%|          | 0/17619 [00:00<?, ? examples/s]

✅ Augmentation counts: {'anger': 2302, 'contempt': 536, 'disgust': 309, 'fear': 1432, 'happiness': 2892, 'neutral': 3334, 'questioning': 1943, 'sadness': 1706, 'surprise': 2779, 'unknown': 386}
✅ Saved augmentation snapshot to /Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V14_20250614_190959/V14_augmentation_snapshot.csv


In [13]:
# --------------------------
# 9. Train-Validation Split
# --------------------------
split_dataset = dataset.train_test_split(test_size=0.2)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

In [14]:
# --------------------------
# 10. Label Distribution Snapshot and Drift Monitor
# --------------------------
snapshot_dir = os.path.join(SAVE_DIR, "label_snapshots")
os.makedirs(snapshot_dir, exist_ok=True)

# Count current training labels
train_label_names = [LABEL_NAMES[i] for i in train_dataset['label']]
label_counts = pd.Series(train_label_names).value_counts().sort_index()
label_counts.name = VERSION

# Save snapshot CSV
snapshot_path = os.path.join(snapshot_dir, f"{VERSION}_label_distribution.csv")
label_counts.to_csv(snapshot_path)
print(f"📊 Saved label distribution snapshot: {snapshot_path}")

# Optionally compare to previous version
previous_versions = sorted([
    f for f in os.listdir(snapshot_dir) if f.endswith(".csv") and not f.startswith(VERSION)
])
if previous_versions:
    latest_prev = previous_versions[-1]
    prev_df = pd.read_csv(os.path.join(snapshot_dir, latest_prev), index_col=0)
    diff = label_counts.subtract(prev_df.iloc[:, 0], fill_value=0)
    print("🔍 Label count change since last snapshot:")
    print(diff)

📊 Saved label distribution snapshot: /Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V14_20250614_190959/label_snapshots/V14_label_distribution.csv


In [15]:
# --------------------------
# 11. Balance Dataset (with NO oversampling for 'unknown')
# --------------------------
mp.set_start_method('fork', force=True)

label_target = 3000
balanced_subsets = []

label_counts = Counter(train_dataset["label"])
print("Original label distribution:", label_counts)

for label, count in label_counts.items():
    subset = train_dataset.filter(lambda x: x['label'] == label, num_proc=1)
    if LABEL_NAMES[label] == "unknown":
        # Just append all 'unknown' examples as-is (no upsampling)
        balanced_subsets.append(subset)
    elif count > label_target:
        subset = subset.select(random.sample(range(len(subset)), label_target))
        balanced_subsets.append(subset)
    elif count < label_target:
        multiplier = label_target // len(subset)
        remainder = label_target % len(subset)
        subset = concatenate_datasets([subset] * multiplier + [subset.select(range(remainder))])
        balanced_subsets.append(subset)
    else:
        balanced_subsets.append(subset)

train_dataset = concatenate_datasets(balanced_subsets).shuffle(seed=42)
print("After balancing:", Counter(train_dataset['label']))

Original label distribution: Counter({4: 2670, 3: 2295, 7: 2213, 0: 1848, 5: 1568, 6: 1346, 2: 1161, 8: 435, 9: 313, 1: 246})


Filter:   0%|          | 0/14095 [00:00<?, ? examples/s]

Filter:   0%|          | 0/14095 [00:00<?, ? examples/s]

Filter:   0%|          | 0/14095 [00:00<?, ? examples/s]

Filter:   0%|          | 0/14095 [00:00<?, ? examples/s]

...

Filter:   0%|          | 0/14095 [00:00<?, ? examples/s]

Filter:   0%|          | 0/14095 [00:00<?, ? examples/s]

Filter:   0%|          | 0/14095 [00:00<?, ? examples/s]

Filter:   0%|          | 0/14095 [00:00<?, ? examples/s]

After balancing: Counter({6: 3000, 7: 3000, 1: 3000, 0: 3000, 3: 3000, 8: 3000, 5: 3000, 4: 3000, 2: 3000, 9: 313})


In [16]:
# --------------------------
# 12. Define Training Arguments for Robust Fine-Tuning
# --------------------------
training_args = TrainingArguments(
    output_dir=SAVE_DIR,                   # Directory to save checkpoints and the final model
    eval_strategy="epoch",                 # Evaluate at the end of each epoch
    save_strategy="epoch",                 # Save checkpoint at each epoch
    save_total_limit=2,                    # ✅ (optional) Keep only last 2 checkpoints to save space
    learning_rate=4e-5,                    # A conservative learning rate for fine-tuning
    per_device_train_batch_size=8,         # Adjust based on your CPU memory limits
    per_device_eval_batch_size=8,
    num_train_epochs=5,                    # Fine-tune for a few epochs (adjust as needed)
    load_best_model_at_end=True,           # Automatically load the best model when training finishes
    metric_for_best_model="accuracy",      # Monitor accuracy for best model selection
    logging_dir=os.path.join(SAVE_DIR, "logs"),  # ✅ Save logs inside versioned folder
    logging_strategy="epoch",                 # ✅ Log once per epoch
    save_safetensors=True                  # ✅ Optional: saves model weights in `.safetensors` (safe format)
)

In [17]:
# --------------------------
# 13. Define Compute Metrics
# --------------------------
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = (predictions == labels).mean()
    return {"accuracy": accuracy}

In [18]:
# --------------------------
# 14. Trainer with Class-Weighted Loss
# --------------------------

# Define custom Trainer to inject class weights
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        # Use smoothed CE + confidence penalty
        smooth_ce_loss = SmoothedCrossEntropyLoss(smoothing=0.05)
        loss = smooth_ce_loss(logits, labels) + confidence_penalty(logits, beta=0.05)
        return (loss, outputs) if return_outputs else loss

# Modify training args for learning rate scheduling and early stopping
training_args.load_best_model_at_end = True
training_args.metric_for_best_model = "eval_loss"
training_args.evaluation_strategy = "epoch"
training_args.save_strategy = "epoch"

# Add EarlyStoppingCallback
early_stop_callback = EarlyStoppingCallback(
    early_stopping_patience=3,
    early_stopping_threshold=0.001
)

# Initialize WeightedTrainer with focal loss, confidence penalty, and label smoothing
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics_with_confusion,
    optimizers=(
        AdamW(model.parameters(), lr=training_args.learning_rate, weight_decay=0.01),
        None
    ),
    callbacks=[early_stop_callback]
)

# T_0 = epochs before first restart, T_mult = restart multiplier
scheduler = CosineAnnealingWarmRestarts(trainer.optimizer, T_0=2, T_mult=2)

# Add scheduler step logic inside the training loop:
original_train = trainer.train

def modified_train(*args, **kwargs):
    result = original_train(*args, **kwargs)
    scheduler.step(trainer.state.epoch)  # instead of eval_loss
    return result

# Fine-tune model
trainer.train()

# Save model
save_model_and_processor(model, processor, SAVE_DIR)

/Users/natalyagrokh/miniconda3/envs/ml_expressions/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy
1,0.574100,0.623898,0.906924
2,0.355900,0.693156,0.907491
3,0.335100,0.708004,0.904938
4,0.327800,0.691474,0.907775
5,0.323700,0.681942,0.911464



Classification Report:
              precision    recall  f1-score   support

       anger       0.91      0.95      0.93       454
     disgust       0.98      0.71      0.83        63
        fear       0.79      0.77      0.78       271
   happiness       0.97      0.98      0.98       597
     neutral       0.94      0.95      0.94       664
 questioning       0.88      0.90      0.89       375
     sadness       0.93      0.85      0.89       360
    surprise       0.93      0.92      0.93       566
    contempt       0.50      0.53      0.52       101
     unknown       0.97      1.00      0.99        73

    accuracy                           0.91      3524
   macro avg       0.88      0.86      0.87      3524
weighted avg       0.91      0.91      0.91      3524


Top 3 confused class pairs:
  - contempt → questioning: 35 instances
  - fear → surprise: 32 instances
  - surprise → fear: 29 instances

🧠 Avg prediction entropy: 0.3464

🔍 Class entropies (sorted):
  - fear: entrop

/Users/natalyagrokh/miniconda3/envs/ml_expressions/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)



Classification Report:
              precision    recall  f1-score   support

       anger       0.90      0.96      0.93       454
     disgust       0.95      0.63      0.76        63
        fear       0.84      0.70      0.76       271
   happiness       0.97      0.98      0.98       597
     neutral       0.93      0.97      0.95       664
 questioning       0.87      0.93      0.90       375
     sadness       0.91      0.85      0.88       360
    surprise       0.91      0.94      0.92       566
    contempt       0.52      0.43      0.47       101
     unknown       1.00      1.00      1.00        73

    accuracy                           0.91      3524
   macro avg       0.88      0.84      0.86      3524
weighted avg       0.90      0.91      0.90      3524


Top 3 confused class pairs:
  - fear → surprise: 43 instances
  - contempt → questioning: 40 instances
  - surprise → fear: 21 instances

🧠 Avg prediction entropy: 0.2888

🔍 Class entropies (sorted):
  - contempt: en

...


Classification Report:
              precision    recall  f1-score   support

       anger       0.90      0.94      0.92       454
     disgust       0.93      0.62      0.74        63
        fear       0.85      0.71      0.77       271
   happiness       0.97      0.98      0.98       597
     neutral       0.89      0.98      0.93       664
 questioning       0.90      0.91      0.91       375
     sadness       0.91      0.83      0.87       360
    surprise       0.91      0.95      0.93       566
    contempt       0.60      0.38      0.46       101
     unknown       1.00      1.00      1.00        73

    accuracy                           0.90      3524
   macro avg       0.89      0.83      0.85      3524
weighted avg       0.90      0.90      0.90      3524


Top 3 confused class pairs:
  - fear → surprise: 45 instances
  - contempt → questioning: 36 instances
  - sadness → neutral: 25 instances

🧠 Avg prediction entropy: 0.2810

🔍 Class entropies (sorted):
  - disgust: e

/Users/natalyagrokh/miniconda3/envs/ml_expressions/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)



Classification Report:
              precision    recall  f1-score   support

       anger       0.94      0.92      0.93       454
     disgust       0.96      0.68      0.80        63
        fear       0.81      0.76      0.78       271
   happiness       0.95      0.99      0.97       597
     neutral       0.91      0.97      0.94       664
 questioning       0.92      0.91      0.92       375
     sadness       0.90      0.83      0.87       360
    surprise       0.92      0.93      0.93       566
    contempt       0.55      0.56      0.56       101
     unknown       1.00      1.00      1.00        73

    accuracy                           0.91      3524
   macro avg       0.89      0.86      0.87      3524
weighted avg       0.91      0.91      0.91      3524


Top 3 confused class pairs:
  - fear → surprise: 36 instances
  - contempt → questioning: 25 instances
  - sadness → neutral: 24 instances

🧠 Avg prediction entropy: 0.2828

🔍 Class entropies (sorted):
  - contempt: 

/Users/natalyagrokh/miniconda3/envs/ml_expressions/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)



Classification Report:
              precision    recall  f1-score   support

       anger       0.91      0.95      0.93       454
     disgust       0.92      0.70      0.79        63
        fear       0.84      0.71      0.77       271
   happiness       0.97      0.98      0.98       597
     neutral       0.92      0.97      0.95       664
 questioning       0.91      0.92      0.92       375
     sadness       0.91      0.84      0.87       360
    surprise       0.91      0.95      0.93       566
    contempt       0.58      0.54      0.56       101
     unknown       1.00      1.00      1.00        73

    accuracy                           0.91      3524
   macro avg       0.89      0.86      0.87      3524
weighted avg       0.91      0.91      0.91      3524


Top 3 confused class pairs:
  - fear → surprise: 45 instances
  - contempt → questioning: 27 instances
  - sadness → neutral: 22 instances

🧠 Avg prediction entropy: 0.2812

🔍 Class entropies (sorted):
  - contempt: 

In [ ]:
# # --------------------------
# # 15. Rescue & Save from Last Checkpoint (after training)
# # --------------------------
# #in case model save fails, resume from latest checkpoint
# processor.save_pretrained(SAVE_DIR)
# print("✅ Processor manually re-saved.")

# # Use parent directory of SAVE_DIR to locate latest V* folder
# parent_dir = os.path.dirname(SAVE_DIR)
# v_folders = [
#     d for d in os.listdir(parent_dir)
#     if os.path.isdir(os.path.join(parent_dir, d)) and d.startswith("V")
# ]

# def extract_timestamp(name):
#     try:
#         _, date_str, time_str = name.split("_")
#         return datetime.strptime(f"{date_str}_{time_str}", "%Y%m%d_%H%M%S")
#     except Exception:
#         return datetime.min

# latest_version_folder = max(v_folders, key=extract_timestamp)
# latest_version_path = os.path.join(parent_dir, latest_version_folder)
# print(f"🗂️ Using latest version folder: {latest_version_path}")

# # Locate latest checkpoint within that version folder
# checkpoint_dirs = [
#     os.path.join(latest_version_path, d)
#     for d in os.listdir(latest_version_path)
#     if d.startswith("checkpoint-") and os.path.isdir(os.path.join(latest_version_path, d))
# ]
# if not checkpoint_dirs:
#     raise ValueError("❌ No checkpoint found in latest version folder.")

# latest_checkpoint = max(checkpoint_dirs, key=os.path.getmtime)
# print(f"✅ Found latest checkpoint: {latest_checkpoint}")

# # Load model and processor from latest checkpoint and save them
# model = AutoModelForImageClassification.from_pretrained(latest_checkpoint)
# processor = AutoImageProcessor.from_pretrained(latest_version_path)
# model = model.to("cpu")

In [19]:
# --------------------------
# 16. Inference Utilities
# --------------------------

# Reload Model for Inference
model = AutoModelForImageClassification.from_pretrained(SAVE_DIR).to(device).eval()
print("✅ Model reloaded for inference.")

# Single image prediction (unbatched)
def predict_label(image_path, threshold=0.85):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(image, return_tensors="pt").to(device)
    with torch.no_grad():
        logits = model(**inputs).logits
        probs = F.softmax(logits, dim=-1)
        conf, pred_idx = torch.max(probs, dim=-1)
    return (id2label[pred_idx.item()], conf.item()) if conf.item() >= threshold else ("REVIEW", conf.item())

# Batched prediction
def batch_predict(image_folder, batch_size=64, threshold=0.85):
    all_preds = []
    error_count = 0
    image_paths = [
        p for p in Path(image_folder).rglob("*")
        if is_valid_image(p.name)
    ]

    for i in tqdm(range(0, len(image_paths), batch_size), desc="Running inference in batches"):
        batch_paths = image_paths[i:i + batch_size]
        images, valid_paths = [], []

        for path in batch_paths:
            try:
                img = Image.open(path).convert("RGB")
                images.append(img)
                valid_paths.append(str(path))
            except Exception:
                error_count += 1
                continue

        if not images:
            continue

        inputs = processor(images=images, return_tensors="pt").to(device)
        with torch.no_grad():
            logits = model(**inputs).logits
            probs = F.softmax(logits, dim=-1)
            confs, preds = torch.max(probs, dim=-1)

        for pred, conf, path in zip(preds.tolist(), confs.tolist(), valid_paths):
            all_preds.append(LABEL_NAMES[pred] if conf >= threshold else "REVIEW")

    print(f"✅ Inference complete. Skipped {error_count} invalid image(s).")
    return all_preds

# Distribution plot
def plot_distribution(predictions, output_path):
    label_counts = Counter(predictions)
    labels = sorted(label_counts.keys())
    counts = [label_counts[label] for label in labels]

    plt.figure(figsize=(10, 5))
    plt.bar(labels, counts)
    plt.title("Predicted Expression Distribution")
    plt.xlabel("Expression")
    plt.ylabel("Count")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(output_path)
    plt.close()

Some weights of the model checkpoint at /Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V14_20250614_190959 were not used when initializing ViTForImageClassification: ['classifier.1.bias', 'classifier.1.weight']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at /Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V14_20250614_190959 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model 

✅ Model reloaded for inference.


In [20]:
# --------------------------
# 17. Entry Point for Inference
# --------------------------
if __name__ == "__main__" and RUN_INFERENCE:

    # Auto-locate latest model directory
    OUTPUT_PATH = os.path.join(SAVE_DIR, f"{VERSION}_distribution_plot_{timestamp}.png")

    predictions = batch_predict(IMAGE_DIR)
    reviewed_paths = []
    image_paths = [str(p) for p in Path(IMAGE_DIR).rglob("*") if is_valid_image(p.name)]

    for path, label in zip(image_paths, predictions):
        if label == "REVIEW":
            reviewed_paths.append(path)

    # Save paths to inspect manually
    with open(os.path.join(SAVE_DIR, f"{VERSION}_review_candidates.txt"), "w") as f:
        f.write("\n".join(reviewed_paths))
    print(f"📝 Saved REVIEW file paths to {VERSION}_review_candidates.txt")

    plot_distribution(predictions, OUTPUT_PATH)
    print(f"Distribution plot saved to: {OUTPUT_PATH}")

Running inference in batches: 100%|███████████| 276/276 [08:00<00:00,  1.74s/it]

✅ Inference complete. Skipped 0 invalid image(s).
📝 Saved REVIEW file paths to V14_review_candidates.txt
Distribution plot saved to: /Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V14_20250614_190959/V14_distribution_plot_20250614_190959.png


In [21]:
# --------------------------
# 18. Temperature Scaling Calibration 
# --------------------------

# Wrapper model for calibrated inference
class ModelWithTemperature(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.temperature = nn.Parameter(torch.ones(1) * 1.5)

    def forward(self, input_ids=None, pixel_values=None, **kwargs):
        logits = self.model(pixel_values=pixel_values).logits
        return logits / self.temperature

    def set_temperature(self, logits, labels):
        nll_criterion = nn.CrossEntropyLoss()
        optimizer = LBFGS([self.temperature], lr=0.01, max_iter=50)

        def eval_fn():
            optimizer.zero_grad()
            loss = nll_criterion(logits / self.temperature, labels)
            loss.backward()
            return loss

        optimizer.step(eval_fn)
        print(f"Optimal temperature (wrapped): {self.temperature.item():.4f}")
        return self

# Dynamically locate the most recent V* folder that contains logits/labels
base_dir = os.path.dirname(SAVE_DIR)
v_folders = sorted([
    d for d in os.listdir(base_dir)
    if os.path.isdir(os.path.join(base_dir, d)) and d.startswith("V")
], key=lambda d: os.path.getmtime(os.path.join(base_dir, d)), reverse=True)

logits_path, labels_path = None, None
for v in v_folders:
    version_tag = v.split('_')[0]
    folder_path = os.path.join(base_dir, v)
    logits_candidate = os.path.join(folder_path, f"logits_eval_{version_tag}.npy")
    labels_candidate = os.path.join(folder_path, f"labels_eval_{version_tag}.npy")
    if os.path.exists(logits_candidate) and os.path.exists(labels_candidate):
        INFER_SAVE_DIR = folder_path
        INFER_VERSION = version_tag
        print(f"📁 Using calibration files from: {SAVE_DIR}")
        logits_path = logits_candidate
        labels_path = labels_candidate
        break

# --------------------------
# Run calibration
# --------------------------
if logits_path and labels_path:
    result = apply_temperature_scaling(logits_path, labels_path)
    if result is not None:
        temperature, logits, labels = result
        plot_reliability_diagram(logits, labels, temperature)
else:
    print(f"⚠️ Skipping temperature scaling and diagram (missing logits or labels in {SAVE_DIR})")

📁 Using calibration files from: /Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V14_20250614_190959
📂 Loading logits from: /Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V14_20250614_190959/logits_eval_V14.npy
📂 Loading labels from: /Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V14_20250614_190959/labels_eval_V14.npy
✅ Optimal temperature: 1.3030
✅ Calibrated Log Loss: 0.4345
📊 Saved reliability diagram to /Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V14_20250614_190959/V14_reliability_diagram_calibrated.png


In [22]:
# --------------------------
# 19. Review & Relabel 'REVIEW' Predictions
# --------------------------
# Section #18 and #19 (REVIEW handling paths)
REVIEW_THRESHOLD = 0.85
REVIEW_BY_CLASS_DIR = os.path.join(SAVE_DIR, "review_predictions_by_class")
REVIEW_CSV_LOG = os.path.join(SAVE_DIR, f"{VERSION}_review_predictions_with_preds.csv")
REVIEW_DIR = os.path.join(SAVE_DIR, "review_predictions_clustered")

# ensure all dirs exist
os.makedirs(REVIEW_BY_CLASS_DIR, exist_ok=True)
os.makedirs(REVIEW_DIR, exist_ok=True)


REVIEW_THRESHOLD = 0.85
REVIEW_BY_CLASS_DIR = os.path.join(SAVE_DIR, "review_predictions_by_class")
REVIEW_CSV_LOG = os.path.join(SAVE_DIR, f"{VERSION}_review_predictions_with_preds.csv")
os.makedirs(REVIEW_BY_CLASS_DIR, exist_ok=True)

REVIEW_SORT_DIR = os.path.join(SAVE_DIR, "review_predictions_by_class")
os.makedirs(REVIEW_SORT_DIR, exist_ok=True)


image_paths = [
    p for p in Path(IMAGE_DIR).rglob("*")
    if p.is_file() and p.suffix.lower() in [".jpg", ".jpeg", ".png"]
]

review_log = []

for img_path in image_paths:
    try:
        image = Image.open(img_path).convert("RGB")
        inputs = processor(image, return_tensors="pt").to(device)
        with torch.no_grad():
            logits = model(**inputs).logits
            probs = F.softmax(logits, dim=-1)
            conf, pred_idx = torch.max(probs, dim=-1)

        conf_val = conf.item()
        pred_label = id2label[pred_idx.item()]
        tag = "REVIEW" if conf_val < REVIEW_THRESHOLD else pred_label

        review_log.append({
            "image_path": str(img_path),
            "predicted_label": pred_label,
            "confidence": round(conf_val, 4),
            "tag": tag
        })

        if tag == "REVIEW":
            target_dir = os.path.join(REVIEW_BY_CLASS_DIR, pred_label)
            os.makedirs(target_dir, exist_ok=True)
            shutil.copy(str(img_path), target_dir)

    except Exception as e:
        print(f"⚠️ Error with image: {img_path} | {e}")

# Save results
pd.DataFrame(review_log).to_csv(REVIEW_CSV_LOG, index=False)
print(f"✅ Completed tagging + copying REVIEW predictions to: {REVIEW_BY_CLASS_DIR}")
print(f"📄 CSV log saved to: {REVIEW_CSV_LOG}")

✅ Completed tagging + copying REVIEW predictions to: /Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V14_20250614_190959/review_predictions_by_class
📄 CSV log saved to: /Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V14_20250614_190959/V14_review_predictions_with_preds.csv


In [23]:
# --------------------------
# 20. Organize REVIEW-tagged Images by Predicted Class
# --------------------------

REVIEW_SORT_DIR = os.path.join(SAVE_DIR, "review_predictions_by_class")
os.makedirs(REVIEW_SORT_DIR, exist_ok=True)

# Section #19 explicitly depends on CSV log generated in Section #16
# Make sure Section #16 ran successfully before proceeding
review_txt_path = os.path.join(SAVE_DIR, f"{VERSION}_review_candidates.txt")
csv_path = os.path.join(SAVE_DIR, f"{VERSION}_review_predictions_with_preds.csv")

if os.path.exists(review_txt_path) and os.path.exists(csv_path):
    with open(review_txt_path, "r") as f:
        review_paths = {line.strip() for line in f.readlines()}

    df = pd.read_csv(csv_path)
    count = 0

    print(f"🔍 Found {len(df)} total predictions (CSV) and {len(review_paths)} REVIEW-tagged paths.")

    for _, row in df.iterrows():
        path = row["image_path"]
        label = row["predicted_label"]
        conf = row["confidence"]

        # ✅ Only process if path was REVIEW-tagged *and* confidence was below threshold
        if path in review_paths and label != "REVIEW":
            dest_dir = os.path.join(REVIEW_SORT_DIR, label)
            os.makedirs(dest_dir, exist_ok=True)
            shutil.copy(path, dest_dir)
            count += 1

    print(f"📂 Grouped {count} REVIEW images into folders by predicted label in: {REVIEW_SORT_DIR}")

else:
    print("⚠️ Missing review candidates file or prediction CSV.")


🔍 Found 17479 total predictions (CSV) and 17620 REVIEW-tagged paths.
📂 Grouped 17479 REVIEW images into folders by predicted label in: /Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V14_20250614_190959/review_predictions_by_class
